In [ ]:
# =============================================================================
# 导入必要的库
# =============================================================================

# 基础文件操作和数据处理
import os
import pandas as pd
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset
import shapefile
import time

# 科学计算
import numpy.ma as ma
import xarray as xr
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

# 可视化
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.font_manager as fm
import seaborn as sns
import cmaps

# 地理信息处理
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.io.img_tiles as cimgt
from cartopy.io.shapereader import Reader
import cartopy.io.shapereader as shpreader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
            
from datetime import datetime

FigurePath = '/home/yfdong/data/work/SMmerge/CN/merge/code/analysis_code/Figure'        

In [ ]:
# 自定义函数
def get_letter(i):
    if 0 <= i < 26:
        return chr(ord('a') + i)
    else:
        return None  # 或者你可以选择抛出异常或返回其他值  

def abbre_to_full(abbreviation):
    # 定义缩写到全称的映射字典
    abbreviation_mapping = {
        "RF": "RandomForest",
        "LGBM": "LightGBM",
        "CB": "CSMX",
        "XGB": "XGBoost",
        # "WHO": "World Health Organization"
    }

    # 查找缩写对应的全称
    full_name = abbreviation_mapping.get(abbreviation.upper())

    if full_name:
        return full_name
    else:
        return f"未找到 '{abbreviation}' 的全称。"
# ---------------------------- 计算时间评分 ---------------------------- 
def calculate_Tcorr(SM_datas, group, SampleLengthThreshold=90):
    obs = group["OBS"]
    corrs = {}
    for col in  SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            # print(f"Warning: Insufficient data for ID {group['ID'].iloc[0]} and column {col}")
            # print(f"Group ID: {group['ID'].iloc[0]}")
            # print(f"Number of observations: {len(obs)}")
            corrs[col] = np.nan
            continue
        corr = np.corrcoef(obs, group[col])[1, 0]
        corrs[col] = corr
    corrs['ID'] = group['ID'].iloc[0]
    return pd.Series(corrs)

# 计算Bias偏差 
def calculate_Tbias(SM_datas, group, SampleLengthThreshold=90):
    obs = group["OBS"]
    # print(f"Group columns: {group.columns}")
    biases = {}
    for col in SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            biases[col] = np.nan
            continue
        bias = np.mean(sim) - np.mean(obs)
        biases[col] = bias
    biases['ID'] = group['ID'].iloc[0]
    return pd.Series(biases)


def calculate_Tkge(SM_datas, group, SampleLengthThreshold=90):
    obs = group["OBS"]
    kges = {}
    # print(f"Group columns: {group.columns}")
    # 调试信息
    for col in SM_datas:
        sim = group[col]
        
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            kges[col] = np.nan
            continue
        
        # 计算相关系数
        r = np.corrcoef(obs, sim)[1, 0]
        alpha = np.mean(sim) / np.mean(obs)
        
        # 处理标准差为零的情况
        if (np.std(obs) == 0):
            beta = np.std(sim) / 0.01
        else:
            beta = np.std(sim) / np.std(obs)
        
        kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
        kges[col] = kge
        # print
        # 保留 ID 列
    kges['ID'] = group['ID'].iloc[0]
        # print(group['ID'].iloc[0])
    return pd.Series(kges)

def calculate_Trmse(SM_datas, group, SampleLengthThreshold=90):
    obs = group["OBS"]
    rmses = {}
    for col in SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            rmses[col] = np.nan
            continue
        rmses[col] = np.sqrt(mean_squared_error(obs, sim))
    rmses['ID'] = group['ID'].iloc[0]
    return pd.Series(rmses) 
# ---------------------------- 计算空间评分 ----------------------------
# 空间评分
def calculate_Scorr(SM_datas, group, SampleLengthThreshold=10):
    obs = group["OBS"]
    corrs = {}
    for col in  SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            corrs[col] = np.nan
            continue
        corr = np.corrcoef(obs, group[col])[1, 0]
        corrs[col] = corr
    corrs['Date'] = group['Date'].iloc[0]
    return pd.Series(corrs)

# 计算Bias偏差 
def calculate_Sbias(SM_datas, group, SampleLengthThreshold=10):
    obs = group["OBS"]
    # print(f"Group columns: {group.columns}")
    biases = {}
    for col in SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            biases[col] = np.nan
            continue
        bias = np.mean(sim) - np.mean(obs)
        biases[col] = bias
    biases['Date'] = group['Date'].iloc[0]
    return pd.Series(biases)


def calculate_Skge(SM_datas, group, SampleLengthThreshold=10):
    obs = group["OBS"]
    kges = {}
    # 调试信息
    for col in SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            kges[col] = np.nan
            continue
        
        # 计算相关系数
        r = np.corrcoef(obs, sim)[1, 0]
        alpha = np.mean(sim) / np.mean(obs)
        
        # 处理标准差为零的情况
        if (np.std(obs) == 0):
            beta = np.std(sim) / 0.01
        else:
            beta = np.std(sim) / np.std(obs)
        
        kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)
        kges[col] = kge
        # print
        # 保留 ID 列
    kges['Date'] = group['Date'].iloc[0]
        # print(group['ID'].iloc[0])
    return pd.Series(kges)
# 计算RMSE
def calculate_Srmse(SM_datas, group, SampleLengthThreshold=10):
    obs = group["OBS"]
    rmses = {}
    for col in SM_datas:
        sim = group[col]
        # 检查数据长度
        if len(obs) < SampleLengthThreshold or len(sim) < SampleLengthThreshold:
            rmses[col] = np.nan
            continue
        rmses[col] = np.sqrt(mean_squared_error(obs, sim))
    rmses['Date'] = group['Date'].iloc[0]
    return pd.Series(rmses)  

# 统计评分的均值（去除异常值）
def calculate_statistics_scipy(dataframe, column_name, iqr_factor=1.5, use_median=False):
    """
    该函数使用 scipy 库去掉极端值后计算平均值或中位数。

    参数:
    dataframe (pd.DataFrame): 包含数据的 DataFrame
    column_name (str): 要处理的列名
    iqr_factor (float): IQR 方法中的系数，默认为 1.5

    返回:
    float: 去掉极端值后的平均值或中位数，保留四位小数。
    """
    if use_median:
        # 计算中位数并保留四位小数
        result = dataframe[column_name].median()#
    else:
        # 计算第一四分位数（Q1）和第三四分位数（Q3）
        Q1 = dataframe[column_name].quantile(0.01)
        Q3 = dataframe[column_name].quantile(0.95)
        IQR = Q3 - Q1
        # print('Q1, Q3:',Q1, Q3)
        # 定义上下边界
        lower_bound = Q1 - iqr_factor * IQR
        upper_bound = Q3 + iqr_factor * IQR

        # 移除极端值
        filtered_df = dataframe[(dataframe[column_name] >= Q1) & (dataframe[column_name] <= Q3)]

        # 计算去掉极端值后的平均值，并保留四位小数
        result = np.round(filtered_df[column_name].mean(),4)
    return result

def get_letter(i):
    if 0 <= i < 26:
        return chr(ord('a') + i)
    else:
        return None  # 或者你可以选择抛出异常或返回其他值  
    
# =============================================================================
# =============================================================================
MODEL_NAME = "CB"
MODEL_FULL_FNAME = abbre_to_full(MODEL_NAME)
SOIL_DEPTH = '10cm'
# 指定文件路径
BASE_PATH = "/home/yfdong/data/work/SMmerge/CN/merge/code/analysis_code/"
DatabasePath = os.path.join(BASE_PATH, 'DataBase')
ModelPath = os.path.join(BASE_PATH, 'Model')
CODE_PATH = os.path.join(BASE_PATH, 'boxplot')
Figure_path = os.path.join(CODE_PATH,"Fig")
# ============================================================================


In [ ]:
VarColorsDict = {"FY*":"#DD9B81", "GDS-Daily":"#6495ED", "GAMSR*":"#FFF68F","CAMSR*":"#CDBE70", 
                 "ERA5": "#63B8FF", "ERA5-Land": "#4682B4", "SMCI*": "#FF7F24", 
                 "GLDAS-Noah": "#9ACD32", "GLDAS-CLSM":"#698B22","CLDAS*": "#CD5555", 
                 MODEL_FULL_FNAME: "#8968CD"}
Total_SM_datas=[ "CAMSR*", "GAMSR*", "GDS-Daily", "FY*", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]


In [ ]:
# 读取数据
# =============================================================================
# 读取数据文件
db_filename = f"merged_test_SM_{MODEL_NAME}_{SOIL_DEPTH}_GDS_Daily_AMSR_FY.csv"
db_file = os.path.join(DatabasePath, db_filename)
db_df = pd.read_csv(db_file)
import geopandas as gpd
# 读取shapefile文件
basin_shp_path = "/home/yfdong/data/map/Basin_shp/"
basin_shp_filename = "China_nine_basin_Project.shp" 
basin_shp = gpd.read_file(os.path.join(basin_shp_path, basin_shp_filename))
# =============================================================================
print(len(db_df))
print(db_df.columns)

In [ ]:
# =============================================================================
# -------------------------------- 计算时间评分 ---------------------------------
id_vars = "ID"
Subset_SM_datas=[ "CAMSR*", "GAMSR*", "GDS-Daily", "FY*", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]

db_df['ID'] = db_df['ID'].astype(str)

ID_df = db_df.groupby('ID').apply(lambda x: x[['ID', 'Latitude', 'Longitude']].iloc[0]).reset_index(drop=True)

Tcorr_df = (
    db_df.groupby(id_vars, as_index=False)
    .apply(lambda group: calculate_Tcorr(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=90))
    .reset_index(drop=True)
)  # 相关系数
Tkge_df = (
    db_df.groupby(id_vars, as_index=False)
    .apply(lambda group: calculate_Tkge(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=90))
    .reset_index(drop=True)
)  # 均方根误差
Tbias_df = (
    db_df.groupby(id_vars, as_index=False)
    .apply(lambda group: calculate_Tbias(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=90))
    .reset_index(drop=True)
)  # 均方根误差
Trmse_df = (
    db_df.groupby(id_vars, as_index=False)
    .apply(lambda group: calculate_Trmse(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=90))
    .reset_index(drop=True)
)  # 均方根误差
# =============================================================================
kge_df = pd.merge(Tkge_df, ID_df, on='ID', how="inner")
rmse_df = pd.merge(Trmse_df, ID_df, on='ID', how="inner")
bias_df = pd.merge(Tbias_df, ID_df, on='ID', how="inner")
corr_df = pd.merge(Tcorr_df, ID_df, on='ID', how="inner")

print("kge_df.columns", kge_df.columns)


In [ ]:

# ------------------------------------绘制箱型图----------------------------------
# 创建一个大的figure对象来容纳所有子图
# 设置图形和子图
title_fontsize = 10
xy_labelsize = 8
fig, axs = plt.subplots(2, 2, figsize=(9, 4), sharex='all', dpi=500)  # 修改为2x2子图布局
# 子图索引
metrics_list = ['KGE', 'BIAS', 'RMSE', 'CORR']
# 循环绘制每个子图
for i, metrics in enumerate(metrics_list):
    row, col = divmod(i, 2)  # 计算行列索引
    ax = axs[row, col]  # 获取当前子图的轴

    # 根据metrics选择数据
    if metrics == 'RMSE':
        data = Trmse_df.melt(id_vars=id_vars, var_name="Variable", value_name=metrics)
        subtitle = 'RMSE(m³/m³)'
    elif metrics == 'CORR':
        data = Tcorr_df.melt(id_vars=id_vars, var_name="Variable", value_name=metrics)
        subtitle = 'CORR'
    elif metrics == 'KGE':
        data = Tkge_df.melt(id_vars=id_vars, var_name="Variable", value_name=metrics)
        subtitle = 'KGE'
    elif metrics == 'BIAS':
        data = Tbias_df.melt(id_vars=id_vars, var_name="Variable", value_name=metrics)
        subtitle = 'BIAS(m³/m³)'
    data = data[~data['Variable'].isin(['Longitude', 'Latitude'])]
    # 
    data['Variable'] = [MODEL_FULL_FNAME if var == MODEL_NAME else var for var in data['Variable']]
    # 绘制箱线图
    sns.boxplot(data=data, x="Variable", y=metrics, hue="Variable", saturation=1, width=0.6, linewidth=0.8, palette=VarColorsDict, 
                showmeans=True,
                meanprops={
                    'marker': 'o',
                    'markerfacecolor': 'white', 
                    'markeredgecolor': 'black',
                    'markersize': 3
                },
                showfliers=True,
                flierprops={
                'marker': 'o',          # 设置异常点的形状
                'markerfacecolor': 'grey',  # 设置异常点的填充颜色
                'markeredgecolor': 'black',# 设置异常点的边框颜色
                'markersize': 0.5,        # 设置异常点的大小
                'alpha': 0.3           # 设置异常点的透明度
                },
                ax=ax)
    
    # 设置标题和标签
    ax.set_title(f"({get_letter(i)}) {subtitle}", fontsize=title_fontsize)
    ax.tick_params(axis='x', labelsize=xy_labelsize)
    ax.tick_params(axis='y', labelsize=xy_labelsize)
    ax.set_xlabel(None)
    ax.set_ylabel(None)
    ax.grid(True, linestyle='--', alpha=0.7)
    if metrics == 'KGE':
        ax.set_ylim(-2, 1)
    elif metrics == 'CORR':
        ax.set_ylim(-0.5, 1)
    elif metrics == 'BIAS':
        ax.set_ylim(-0.3,0.3)
    elif metrics =='RMSE':
        ax.set_ylim(0,0.3)
        
    # 设置x轴刻度标签
    ax.set_xticks(range(len(data["Variable"].unique())))
    ax.set_xticklabels(data["Variable"].unique(), rotation=45, ha='right', fontsize=xy_labelsize, color='black')
plt.subplots_adjust(wspace=0.15,hspace=0.2)


In [ ]:
Subset_SM_datas=[ "CAMSR*", "GAMSR*", "GDS-Daily", "FY*", "ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]# SM_datas=[ "CAMSR*", "GAMSR*", "GDS_Daily", "FY", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]

Subset_SM_datas=[ "CAMSR*", "GAMSR*",  "FY*",   "SMCI*" ,"CLDAS*" ,MODEL_NAME]# SM_datas=[ "CAMSR*", "GAMSR*", "GDS_Daily", "FY", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]
Rows = len(Subset_SM_datas)

#     return result
def add_text(ax, text):
    # 创建文本框，将文本置于文本框内
    x , y = 0.4 , 0.08
    bbox = {"facecolor": "white", "alpha": 0.6, "edgecolor": "none"}
    styles = {"size": text_size-1, "color": "black", "bbox": bbox, "weight": "normal"}
    ax.text(x, y, text, ha="right", **styles ,transform = ax.transAxes)
    pass
# =============================================================================
RMSE_df = rmse_df.copy()
CORR_df = corr_df.copy()
KGE_df = kge_df.copy()
BIAS_df = bias_df.copy()
# =============================================================================
# # --------------------可视化-----------------------------

proj = ccrs.PlateCarree() 
fig, axs = plt.subplots(Rows, 4, figsize=(8.5*1., 8*1.05/7*Rows*1.), dpi=300, subplot_kw={'projection': ccrs.PlateCarree()})

axis_labelsize = 9
xy_labelsize = 9
cbar_lablesize = 7
plt.rcParams['font.size'] = 10
text_size = 9
title_size = 9

METRICs = ["KGE", "RMSE", "CORR","BIAS"]
cmap = cmaps.MPL_StepSeq_r

FRAC=0.6
# cmap = cmaps.MPL_StepSeq25_r
for j, METRIC in enumerate(METRICs):
    for i, VAR in enumerate(Subset_SM_datas):
        # print(i,j,VAR,METRIC)
        if METRIC == "RMSE":
            dataframe = RMSE_df
            cmap = cmaps.percent_11lev
        elif METRIC == "CORR":
            dataframe = CORR_df
            cmap = cmaps.percent_11lev
        elif METRIC == "BIAS":
            dataframe = BIAS_df
            cmap = cmaps.CBR_coldhot
        elif METRIC == "KGE":
            dataframe = KGE_df
            cmap = cmaps.percent_11lev
              
        # METRICvalue= calculate_statistics_scipy(dataframe, VAR, iqr_factor=1.5, use_median=False)
        METRICvalue= calculate_statistics_scipy(dataframe, VAR,  use_median=False)
        # print(METRICvalue)
        dataframe= dataframe.sample(frac=FRAC)    
        ax = axs[i,j]
        # =============================================================================
        # 设置图片大小
        proj = ccrs.PlateCarree()  # 创建投影
        # 设置绘图区背景颜色
        ax.set_facecolor('#F5F5F5')
        # 添加海洋淡蓝色填充，以及海岸线
        ax.add_feature(cfeature.OCEAN, zorder=0, facecolor='lightblue')
        ax.add_feature(cfeature.COASTLINE, zorder=0, edgecolor='black', linewidth=0.5)
        # -----------------------------绘制散点图-------------------------------
        # 设置色标
        if METRIC == "CORR":
            vmin, vmax = 0,1
        if METRIC == "RMSE":
            vmin, vmax = 0.02, 0.1
        if METRIC == "KGE":
            vmin, vmax = 0, 0.8
        if METRIC == "BIAS":
            vmin, vmax = -0.1,0.1
            
        if METRIC =="CORR":
            ticks = np.linspace(0., 1, 5)
        elif METRIC =="RMSE":
            ticks = np.linspace(0.02, 0.1, 5)
        elif METRIC =="BIAS":
            ticks = np.linspace(-0.1, 0.1, 5)
        elif METRIC =="KGE":
            ticks = np.linspace(0.0, 0.8, 5)
        
        # 绘制土壤湿度散点图
        scatters = ax.scatter(dataframe["Longitude"], dataframe["Latitude"],  
                              s=0.2, c=dataframe[VAR], alpha=0.8, marker='o', 
                              cmap=cmap, label="SM station", vmin=vmin, vmax=vmax)
        
        # 获取Shapefile文件的投影信息
        shp_crs = basin_shp.crs
        # 设置投影
        proj = ccrs.Projection(shp_crs)
        # 添加流域边界
        basin_shp.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=0.5, alpha=0.7) 
        # =============================================================================
        # 设置绘图范围为中国陆地的经纬度范围
        ax.set_xlim(70, 138)
        ax.set_ylim(15, 56)
        # # =============================================================================
        # 添加文本
        # 创建文本框，将文本置于文本框内
        bbox = {"facecolor": "white", "alpha": 0.5, "edgecolor": "none"}
        # 所有文本使用统一的样式
        styles = {"size": text_size-0.5, "color": "black", "bbox": bbox}
        x , y = 0.05 , 0.35
        text = f"{np.round(METRICvalue,4)}"
        # ----------------归一化坐标------------------
        add_text(ax, text)
        # =============================================================================   
        if i==0:
            title_name = METRIC
            ax.set_title(title_name, fontsize=title_size)
        if j==0:
            # ax.set_ylabel(VAR, fontsize= xy_labelsize)
            VAR = MODEL_FULL_FNAME if VAR == MODEL_NAME else VAR
            ax.text(-0.1, 0.5, VAR, va='center', ha='center', rotation='vertical', 
                    transform=ax.transAxes, fontsize=xy_labelsize)
        ax.set_ylabel(VAR, fontsize= xy_labelsize)
        # if j==4:
        # ----------------------------绘制colorbar----------------------------
        cbar = fig.colorbar(scatters, shrink=0.8, pad=0.05, extend='both', orientation='vertical', ax=ax)#, ticks=levels)
        cbar.ax.tick_params(which='both', length=1.5)
        # 设置colorbar刻度大小
        cbar.set_ticks(ticks)
        cbar.ax.tick_params(labelsize = cbar_lablesize)
        cbar.minorticks_on
        # 添加字母
        idx = len(METRICs)*i+j

# 调整子图间距
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95,wspace=0.08,hspace=0.1)
plt.show()


In [ ]:
# -------------------------------------- 读取流域划分数据 ---------------------------------
Total_Basin_db_filename = f"Total_merged_test_SM_withBASIN_{MODEL_NAME}_{SOIL_DEPTH}_GDS_Daily.h5"
Total_Basin_db_file = os.path.join(DatabasePath, Total_Basin_db_filename)
Total_Basin_db_df = pd.read_hdf(Total_Basin_db_file, key='data')
print(Total_Basin_db_df.columns)
Total_Basin_db_df
print(Total_Basin_db_df['Date'])

Total_Basin_db_df['Date'] = pd.to_datetime(Total_Basin_db_df['Date'])

In [ ]:
# Total_Basin_db_df = Total_Basin_db_df[Total_Basin_db_df['LST']>273]
cols_SM = ['CLDAS_SM', 'GLDAS_Noah_SM',
       'ERA5_Land_SM', 'ERA5_SM', 'SMCI1km_SM', 'SMC_SM', 'GDS_SM', 'CB_grid',
    'GLDAS_CLSM', 'GDS_Daily_SM', 'OBS_SM_10cm']
for col_SM in cols_SM:    
    Total_Basin_db_df.loc[(Total_Basin_db_df[col_SM] < 0) | (Total_Basin_db_df[col_SM] > 0.7) , col_SM] = np.nan
    # print(col_SM, np.max(final_data[col_SM]))
# --------土壤湿度--------
for col_SM in cols_SM:     
    Total_Basin_db_df[col_SM].fillna(Total_Basin_db_df[col_SM].median(),inplace=True)

In [ ]:
Total_Numeric_Columns = Total_Basin_db_df.select_dtypes(include=[np.number]).columns
id_vars = 'Date'
# 定义开始和结束日期
start_date = '2016-02-01'
end_date = '2017-10-31'
# 设置图片参数
axplot_linewidth = 0.8
axplot_alpha = 0.5
fill_alpha = 0.2
alpha_plot = 0.5
linewidth_plot = 0.6

title_size = 7
xy_labelsize = 6
legend_size =7
SCALEFACTOR = 1
# 筛选出指定日期范围内的数据
filtered_Total_Basin_db_df = Total_Basin_db_df[(Total_Basin_db_df['Date'] >= start_date) & (Total_Basin_db_df['Date'] <= end_date)]

BASINS = ["Song-Liao","Inland","Yellow","Haihe", "Huaihe", "Yangtze", "Southwest",  "Southeast", "Pearl" ,"CN"]
# 为每个流域设置独立的 y 轴范围
ylim_dict = {
    "Song-Liao": (0.05, 0.4),
    "Inland": (0.1, 0.3),
    "Yellow": (0.1, 0.4),
    "Haihe": (0.1, 0.4),
    "Huaihe": (0.15, 0.4),
    "Yangtze": (0.14, 0.45),
    "Southwest": (0.1, 0.45),
    "Southeast": (0.15, 0.5),
    "Pearl": (0.2, 0.5),
    "CN": (0.15, 0.35)
}
num_cols = 2
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator 
fig, axs = plt.subplots(nrows=5, ncols=num_cols, figsize=(10*0.9, 9*0.9), sharex='all', dpi=300) # , sharey='all'
for idx, Basin in enumerate(BASINS):
    # 将idx索引转换为2D形式的(row, col)
    row = idx // num_cols
    col = idx % num_cols    
    ax = axs[row, col]
    
    if Basin=="CN":
        subBasin_df = filtered_Total_Basin_db_df.copy()
    else:
        subBasin_df = filtered_Total_Basin_db_df[filtered_Total_Basin_db_df['BASIN']==Basin]

    
    MEAN_subBasin_df = subBasin_df.groupby('Date')[Total_Numeric_Columns].mean().reset_index()
    MEAN_subBasin_df['Date'] = pd.to_datetime(MEAN_subBasin_df['Date'])
    

    # MEAN_subBasin_df["Date"] = pd.date_range(start=start_date, end=end_date, freq='D')   
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"CLDAS_SM"].values *SCALEFACTOR, color = VarColorsDict["CLDAS*"], linewidth = axplot_linewidth*1. , alpha=1, label="CLDAS*")    
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GLDAS_Noah_SM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-Noah"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GLDAS-Noah") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GLDAS_CLSM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-CLSM"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GLDAS-CLSM") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"ERA5_Land_SM"].values *SCALEFACTOR, color = VarColorsDict["ERA5-Land"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="ERA5-Land")
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GDS_Daily_SM"].values *SCALEFACTOR, color = VarColorsDict["GDS-Daily"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GDS-Daily")
    # ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GAMSR*"].values *SCALEFACTOR, color = VarColorsDict["GAMSR*"], linewidth = axplot_linewidth , label="GAMSR*") 
    # ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"CAMSR*"].values *SCALEFACTOR, color = VarColorsDict["CAMSR*"], linewidth = axplot_linewidth , label="CAMSR*") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"SMCI1km_SM"].values *SCALEFACTOR, color = VarColorsDict["SMCI*"], linewidth = axplot_linewidth*1. , alpha=1, label="SMCI*") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df['CB_grid'].values *SCALEFACTOR, color ='#5D478B', linewidth = axplot_linewidth*1.1 , label=MODEL_NAME)
     
    # 绘制观测数据的散点图
    ax.scatter(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"OBS_SM_10cm"].values, 
               facecolors='#CFCFCF', edgecolors= "#1C1C1C", 
               s = 1, alpha = 0.7, label="OBS") #,zorder=10   
    
    # 添加Title
    if Basin=="CN":
        title_name = "Chinese mainland"
    else:
        title_name = f'{Basin}'
        
    ax.set_title(title_name, fontsize= title_size)
    # 设置x轴和y轴数字大小
    ax.tick_params(axis='x', labelsize=xy_labelsize)
    ax.tick_params(axis='y', labelsize=xy_labelsize)

    # 调整刻度线参数：
    ax.minorticks_on()
    # 显示副刻度线
    ax.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
    # 设置 Y 轴范围
    ax.set_ylim(ylim_dict[Basin])  # 为每个流域设置独立的 y 轴范围
    # grid 设置网格线性
    ax.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha = 0.3)

    ax.yaxis.set_major_locator(MaxNLocator(nbins=4, prune='both'))
    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    # 设置主要刻度格式化器为"Month-Year"格式
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # daily
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%B\n%Y'))
    # ax.xaxis.set_major_locator(mdates.DayLocator(interval=Xinterval))  # daily
    
# 添加总的legend
colors = [VarColorsDict["GDS-Daily"], "#4682B4", "#8DEEEE", "#6E8B3D", "#FF7F24", "#CD5555",  "#5D478B", "#1C1C1C"]
labels = [ "GSD-Daily", "ERA5-Land", "GLDAS-CLSM", "GLDAS-Noah", "SMCI*", "CLDAS*", "CSMX", "OBS"]
markers = ['_', '_', '_', '_', '_', '_', '_','.']  # 设置 OBS 为 'o'，其余为 '_'
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
# 创建图例的handles
legend_handles = [mlines.Line2D([], [], color=color, marker=marker, linestyle='None',
                                markersize=6, markeredgewidth=2, label=label)
                  for color, label, marker in zip(colors, labels, markers)]
# 设置字体属性
fontprops = fm.FontProperties(size= legend_size)

# 绘制图例
legend = fig.legend(handles=legend_handles, bbox_to_anchor=(-1.1, -0.45, 2., 0.1),
                    loc='lower center', bbox_transform=axs[4,1].transAxes,
                    prop=fontprops, ncol=9, mode='expand', columnspacing=0.01,frameon=False,
                    labelspacing=0.5 ,# 图例项之间的垂直间距, ,
                    handletextpad=0.2,  # 图例标记与文本之间的间距
                )
# 调整子图间距
plt.subplots_adjust(wspace=0.08,hspace=0.2)
    

In [ ]:
# 定义开始和结束日期
start_date = '2011-02-01'
end_date = '2017-10-31'
# 设置图片参数
axplot_linewidth = 0.6
axplot_alpha = 0.5
fill_alpha = 0.2
alpha_plot = 0.5
linewidth_plot = 0.6

title_size = 7
xy_labelsize = 7
legend_size =7
SCALEFACTOR = 1
# 筛选出指定日期范围内的数据
filtered_Total_Basin_db_df = Total_Basin_db_df[(Total_Basin_db_df['Date'] >= start_date) & (Total_Basin_db_df['Date'] <= end_date)]

BASINS = ["Song-Liao","Inland","Yellow","Haihe", "Huaihe", "Yangtze", "Southwest",  "Southeast", "Pearl" ,"CN"]
# 为每个流域设置独立的 y 轴范围
ylim_dict = {
    "Song-Liao": (0.05, 0.4),
    "Inland": (0.05, 0.4),
    "Yellow": (0.1, 0.4),
    "Haihe": (0.1, 0.4),
    "Huaihe": (0.15, 0.4),
    "Yangtze": (0.15, 0.45),
    "Southwest": (0.1, 0.45),
    "Southeast": (0.15, 0.5),
    "Pearl": (0.1, 0.5),
    "CN": (0.15, 0.35)
}
num_cols = 2
from matplotlib.ticker import MaxNLocator
fig, axs = plt.subplots(nrows=5, ncols=num_cols, figsize=(10*1., 9*1.), sharex='all', dpi=300) # , sharey='all'
for idx, Basin in enumerate(BASINS):
    # 将idx索引转换为2D形式的(row, col)
    row = idx // num_cols
    col = idx % num_cols    
    ax = axs[row, col]
    
    if Basin=="CN":
        subBasin_df = filtered_Total_Basin_db_df.copy()
    else:
        subBasin_df = filtered_Total_Basin_db_df[filtered_Total_Basin_db_df['BASIN']==Basin]

    
    MEAN_subBasin_df = subBasin_df.groupby('Date')[Total_Numeric_Columns].mean().reset_index()
    MEAN_subBasin_df['Date'] = pd.to_datetime(MEAN_subBasin_df['Date'])
    

    # MEAN_subBasin_df["Date"] = pd.date_range(start=start_date, end=end_date, freq='D')   
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"CLDAS_SM"].values *SCALEFACTOR, color = VarColorsDict["CLDAS*"], linewidth = axplot_linewidth*1. , alpha=1, label="CLDAS*")    
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GLDAS_Noah_SM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-Noah"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GLDAS-Noah") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GLDAS_CLSM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-CLSM"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GLDAS-CLSM") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"ERA5_Land_SM"].values *SCALEFACTOR, color = VarColorsDict["ERA5-Land"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="ERA5-Land")
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GDS_Daily_SM"].values *SCALEFACTOR, color = VarColorsDict["GDS-Daily"], linewidth = axplot_linewidth*0.8 , alpha=axplot_alpha, label="GDS-Daily")
    # ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"GAMSR*"].values *SCALEFACTOR, color = VarColorsDict["GAMSR*"], linewidth = axplot_linewidth , label="GAMSR*") 
    # ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"CAMSR*"].values *SCALEFACTOR, color = VarColorsDict["CAMSR*"], linewidth = axplot_linewidth , label="CAMSR*") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"SMCI1km_SM"].values *SCALEFACTOR, color = VarColorsDict["SMCI*"], linewidth = axplot_linewidth*1. , alpha=1, label="SMCI*") 
    ax.plot(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df['CB_grid'].values *SCALEFACTOR, color ='#5D478B', linewidth = axplot_linewidth*1. , label=MODEL_NAME)
     
    # 绘制观测数据的散点图
    ax.scatter(MEAN_subBasin_df["Date"].values, MEAN_subBasin_df[f"OBS_SM_10cm"].values, facecolors='#CFCFCF', edgecolors= "#1C1C1C", s = 1, alpha = 0.7, label="OBS") #,zorder=10   
    
    # 添加Title
    if Basin=="CN":
        title_name = "Chinese mainland"
    else:
        title_name = f'{Basin}'
        
    ax.set_title(title_name, fontsize= title_size)
    # 设置x轴和y轴数字大小
    ax.tick_params(axis='x', labelsize=xy_labelsize)
    ax.tick_params(axis='y', labelsize=xy_labelsize)
    # 调整刻度线参数：
    ax.minorticks_on()
    # 显示副刻度线
    ax.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
    # 设置 Y 轴范围
    ax.set_ylim(ylim_dict[Basin])  # 为每个流域设置独立的 y 轴范围
    # grid 设置网格线性
    ax.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha = 0.3)
    ax.yaxis.set_major_locator(MaxNLocator(nbins=4, prune='both'))
    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    # 设置主要刻度格式化器为"Month-Year"格式
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    # ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # daily
# 添加总的legend
colors = ["#CDBE70", "#4682B4", "#8DEEEE", "#6E8B3D", "#FF7F24", "#CD5555",  "#5D478B", "#1C1C1C"]
labels = [ "GSD-Daily", "ERA5-Land", "GLDAS-CLSM", "GLDAS-Noah", "SMCI*", "CLDAS*", "CSMX", "OBS"]
markers = ['_', '_', '_', '_', '_', '_', '_','.']  # 设置 OBS 为 'o'，其余为 '_'
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
# 创建图例的handles
legend_handles = [mlines.Line2D([], [], color=color, marker=marker, linestyle='None',
                                markersize=6, markeredgewidth=2, label=label)
                  for color, label, marker in zip(colors, labels, markers)]
# 设置字体属性
fontprops = fm.FontProperties(size= legend_size)

# 绘制图例
legend = fig.legend(handles=legend_handles, bbox_to_anchor=(-1.1, -0.3, 2., 0.1),
                    loc='lower center', bbox_transform=axs[4,1].transAxes,
                    prop=fontprops, ncol=9, mode='expand', columnspacing=0.01,frameon=False,
                    labelspacing=0.5 ,# 图例项之间的垂直间距, ,
                    handletextpad=0.2,  # 图例标记与文本之间的间距
                )
# 调整子图间距
plt.subplots_adjust(wspace=0.08,hspace=0.2)
    

In [ ]:
# -------------------------------------- 读取流域划分数据 ---------------------------------
Test_Basin_db_filename = f"merged_test_SM_withBASIN_{MODEL_NAME}_{SOIL_DEPTH}_GDS_Daily.csv"
Test_Basin_db_file = os.path.join(DatabasePath  ,Test_Basin_db_filename)
Test_Basin_db_df = pd.read_csv(Test_Basin_db_file)
print(Test_Basin_db_df.columns)
Test_Basin_db_df
print(Test_Basin_db_df['Date'])
Test_Basin_db_df['Date'] = pd.to_datetime(Test_Basin_db_df['Date'])

In [ ]:
# ----------------------------------- 流域尺度空间评分 ------------------------------------
# 设置参与评估的土壤湿度数据
Subset_SM_datas=[ "GDS-Daily", "ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]# SM_datas=[ "CAMSR*", "GAMSR*", "GDS_Daily", "FY", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]
VarColorsDict = {"FY*":"#DD9B81", "GDS-Daily":"#838B8B", "GAMSR*":"#FFF68F","CAMSR*":"#CDBE70", 
                 "ERA5": "#63B8FF", "ERA5-Land": "#4682B4", "SMCI*": "#FF7F24", 
                 "GLDAS-Noah": "#9ACD32", "GLDAS-CLSM":"#698B22","CLDAS*": "#CD5555", 
                 MODEL_FULL_FNAME: "#8968CD"}
BASINS = ["Song-Liao","Inland","Yellow","Haihe", "Huaihe", "Yangtze", "Southwest",  "Southeast", "Pearl" ,"CN"]
# 逐流域计算
for letterIdx,METRIC in enumerate(['CORR','RMSE']): #KGE, CORR, RMSE, BIAS
# for letterIdx,METRIC in enumerate(['KGE','BIAS']): #KGE, CORR, RMSE, BIAS

    fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(6, 9), sharex='all', sharey='all', dpi=300) #
    for idx, Basin in enumerate(BASINS):
        
        # 将idx索引转换为2D形式的(row, col)
        row = idx // num_cols
        col = idx % num_cols
        ax = axs[row, col]  
        
        if Basin=="CN":
            SM_df = Test_Basin_db_df.copy()
        else:
            SM_df = Test_Basin_db_df[Test_Basin_db_df['BASIN']==Basin]

        # # --------------------------计算空间评分------------------------------
        if METRIC=='RMSE':
            Srmse_df = (
                        SM_df.groupby(id_vars, as_index=False)
                        .apply(lambda group: calculate_Srmse(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=10))
                        .reset_index(drop=True)
                        )  # 均方根误差
            METRICdf =  Srmse_df.copy() 
        elif METRIC=='CORR':
            Scorr_df = (
                        SM_df.groupby(id_vars, as_index=False)
                        .apply(lambda group: calculate_Scorr(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=10))
                        .reset_index(drop=True)
                        )  # 相关系数
            METRICdf =  Scorr_df.copy() 
        elif METRIC=='BIAS':
            Sbias_df = (
                        SM_df.groupby(id_vars, as_index=False)
                        .apply(lambda group: calculate_Sbias(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=10))
                        .reset_index(drop=True)
                        )  # 偏差
            METRICdf =  Sbias_df.copy() 
        elif METRIC=="KGE":

            Skge_df = (
                        SM_df.groupby(id_vars, as_index=False)
                        .apply(lambda group: calculate_Skge(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=10))
                        .reset_index(drop=True)
                        )  # KGE指数
            # 将KGE结果保存到 METRICdf
            METRICdf = Skge_df.copy()

        # 处理为日平均 
        # METRICdf = METRICdf.drop('ID',axis=1)  
        METRICdf['Date'] = pd.to_datetime(METRICdf['Date']).dt.normalize()  # 将 'date' 列转换为 datetime 类型（如果还未转换）
        # METRICdf['Date'] = METRICdf['Date'].dt.strftime('%Y-%m-%d')
        # 
        # 添加月份和日期列
        METRICdf['Month-Day'] = METRICdf['Date'].dt.strftime('%m-%d')
        
        # 按月份和日期分组并计算均值
        METRICdf = METRICdf.drop(columns=['Date']).groupby('Month-Day').mean().reset_index()

        METRICdf['Date'] = pd.to_datetime('2016-' + METRICdf['Month-Day'], format='%Y-%m-%d')
        # METRICdf["Date"] = pd.date_range(start="2016-01-02", end="2016-12-05", freq='D')
        # 设置绘图区背景颜色
        ax.set_facecolor('#F5F5F5')
        if METRIC=="CORR" or METRIC=="RMSE" or METRIC=="KGE":
            # 绘制折线图
            ax.plot(METRICdf["Date"].values, METRICdf[f"CLDAS*"].values *SCALEFACTOR, color = VarColorsDict["CLDAS*"], linewidth = axplot_linewidth*1.2 , label="CLDAS*")    
            ax.plot(METRICdf["Date"].values, METRICdf[f"GLDAS-Noah"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-Noah"], linewidth = axplot_linewidth , label="GLDAS-Noah") 
            ax.plot(METRICdf["Date"].values, METRICdf[f"GLDAS-CLSM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-CLSM"], linewidth = axplot_linewidth , label="GLDAS-CLSM") 
            ax.plot(METRICdf["Date"].values, METRICdf[f"ERA5-Land"].values *SCALEFACTOR, color = VarColorsDict["ERA5-Land"], linewidth = axplot_linewidth , label="ERA5-Land")
            ax.plot(METRICdf["Date"].values, METRICdf[f"GDS-Daily"].values *SCALEFACTOR, color = VarColorsDict["GDS-Daily"], linewidth = axplot_linewidth , label="GDS-Daily")
            ax.plot(METRICdf["Date"].values, METRICdf[f"SMCI*"].values *SCALEFACTOR, color = VarColorsDict["SMCI*"], linewidth = axplot_linewidth , label="SMCI*") 
            ax.plot(METRICdf["Date"].values, METRICdf[MODEL_NAME].values *SCALEFACTOR, color = VarColorsDict[MODEL_FULL_FNAME], linewidth = axplot_linewidth*1.5 , label=MODEL_NAME) # purple
            
        elif METRIC=="BIAS":
            # 面积图
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"CLDAS*"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["CLDAS*"], facecolor = VarColorsDict["CLDAS*"], linewidth = axplot_linewidth*1.5 , alpha = fill_alpha, label="CLDAS*")
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"GLDAS-Noah"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["GLDAS-Noah"], facecolor = VarColorsDict["GLDAS-Noah"], linewidth = axplot_linewidth , alpha = fill_alpha, label="GLDAS-Noah")
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"GLDAS-CLSM"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["GLDAS-CLSM"], facecolor = VarColorsDict["GLDAS-CLSM"], linewidth = axplot_linewidth , alpha = fill_alpha, label="GLDAS-CLSM")
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"ERA5-Land"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["ERA5-Land"] ,facecolor = VarColorsDict["ERA5-Land"], linewidth = axplot_linewidth , alpha = fill_alpha, label="ERA5-Land")
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"SMCI*"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["SMCI*"] ,facecolor = VarColorsDict["SMCI*"], linewidth = axplot_linewidth, alpha = fill_alpha, label="SMCI1km")
            ax.fill_between(METRICdf["Date"].values, METRICdf[f"GDS-Daily"].values*SCALEFACTOR , 0, edgecolor = VarColorsDict["GDS-Daily"] ,facecolor = VarColorsDict["GDS-Daily"], linewidth = axplot_linewidth, alpha = fill_alpha, label="GDS-Daily")
            ax.fill_between(METRICdf["Date"].values, METRICdf[MODEL_NAME].values*SCALEFACTOR , 0, edgecolor = VarColorsDict[MODEL_FULL_FNAME] ,facecolor = VarColorsDict[MODEL_FULL_FNAME], linewidth =  axplot_linewidth*2 , alpha = 0.8, label="RF") #purple     
            # 绘制折线图
            ax.plot(METRICdf["Date"].values, METRICdf[f"CLDAS*"].values *SCALEFACTOR, color = VarColorsDict["CLDAS*"], linewidth = axplot_linewidth*1.2 , label="CLDAS*")    
            ax.plot(METRICdf["Date"].values, METRICdf[f"GLDAS-Noah"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-Noah"], linewidth = axplot_linewidth , label="GLDAS-Noah") 
            ax.plot(METRICdf["Date"].values, METRICdf[f"GLDAS-CLSM"].values *SCALEFACTOR, color = VarColorsDict["GLDAS-CLSM"], linewidth = axplot_linewidth , label="GLDAS-CLSM") 
            ax.plot(METRICdf["Date"].values, METRICdf[f"ERA5-Land"].values *SCALEFACTOR, color = VarColorsDict["ERA5-Land"], linewidth = axplot_linewidth , label="ERA5-Land")
            ax.plot(METRICdf["Date"].values, METRICdf[f"GDS-Daily"].values *SCALEFACTOR, color = VarColorsDict["GDS-Daily"], linewidth = axplot_linewidth , label="GDS-Daily")
            ax.plot(METRICdf["Date"].values, METRICdf[f"SMCI*"].values *SCALEFACTOR, color = VarColorsDict["SMCI*"], linewidth = axplot_linewidth , label="SMCI*") 
            ax.plot(METRICdf["Date"].values, METRICdf[MODEL_NAME].values *SCALEFACTOR, color = VarColorsDict[MODEL_FULL_FNAME], linewidth = axplot_linewidth*1.5 , label=MODEL_NAME) # purple
    
        if idx >=8 :
            # 设置横坐标刻度格式
            ax.xaxis.set_major_locator(mdates.DayLocator(interval=30))
            ax.xaxis.set_major_formatter(mdates.DateFormatter("%m"))
            # 设置横坐标范围
            # start_date = CORRdf["Date"].iloc[0]
            # end_date = CORRdf["Date"].iloc[-1]
            start_date = datetime(2016,4,1)
            end_date = datetime(2016,11,1)
            ax.set_xlim(start_date, end_date)
            
        else:
            ax.tick_params(axis='x', labelbottom=False)
    
        # 添加Title
        if Basin=="CN":
            title_name = "Chinese mainland"
        else:
            title_name = f'{Basin}'
        ax.set_title(title_name, fontsize= title_size)
        # 设置x轴和y轴数字大小
        ax.tick_params(axis='x', labelsize=xy_labelsize-0.5)
        ax.tick_params(axis='y', labelsize=xy_labelsize-0.5)
        
    
        # 调整刻度线参数：
        ax.minorticks_on()
        # 显示副刻度线
        ax.tick_params(axis="both", which="major", direction="in", width=0.9, length=5, color='#363636')
        # grid 设置网格线性
        ax.grid(True, which="major", linestyle="--", color="gray", linewidth=0.75, alpha = 0.3)
        
        # 设置坐标轴范围
        if METRIC=="CORR":
            plt.ylim(-0.1,0.9)
            ax.set_yticks(np.arange(-0.2, 1.2,0.4))
        elif METRIC=="BIAS":
            plt.ylim(-0.07,0.17)
            ax.set_yticks([-0.05, 0, 0.05, 0.1 ,0.15])
        elif METRIC=="RMSE":
            plt.ylim(0,0.21)
            ax.set_yticks([0.05, 0.1, 0.15, 0.2 ])
        elif METRIC=="KGE":
            plt.ylim(-0.5,0.9)
            ax.set_yticks([-0.4,  0, 0.4 , 0.8])
            
    # 添加字母：
    title =  plt.suptitle(f"({get_letter(letterIdx)}) {METRIC}", fontsize=title_size, y=0.99)

    # 调整子图间距
    plt.subplots_adjust(left=0.2 , bottom=0.5,right=0.8,top=0.96,wspace=0.1,hspace=0.4)

    # Save Files    
    if not os.path.exists(Figure_path):
            os.makedirs(Figure_path) 
    
    plt.show()
    plt.close() 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import font_manager as fm
VarColorsDict = {"FY*":"#DD9B81", "GDS-Daily":"#838B8B", "GAMSR*":"#FFF68F","CAMSR*":"#CDBE70", 
                 "ERA5": "#63B8FF", "ERA5-Land": "#4682B4", "SMCI*": "#FF7F24", 
                 "GLDAS-Noah": "#9ACD32", "GLDAS-CLSM":"#698B22","CLDAS*": "#CD5555", 
                 MODEL_FULL_FNAME: "#8968CD"}

# 定义颜色和标签
colors = [ "#838B8B", "#4682B4", "#9ACD32", "#698B22", "#FF7F24", "#CD5555",  "#5D478B"]
labels = [ "GDS-Daily","ERA5-Land","GLDAS-Noah", "GLDAS-CLSM",  "SMCI*", "CLDAS*", 'CSMX']
# 创建图例的handles
legend_handles = [mlines.Line2D([], [], color=color, linestyle='-',linewidth=5,
                                markersize=6, label=label) for color, label in zip(colors, labels)]

# 设置字体属性
legend_size = 12  # 假设一个字体大小，可根据需要调整
fontprops = fm.FontProperties(size=legend_size-1)

# 创建一个新的图形和坐标轴
fig = plt.figure(figsize=(8.5, 0.3), dpi=350)  # 根据需要调整图形大小
ax = fig.add_subplot(111)
ax.axis('off')  # 隐藏坐标轴

# 绘制图例
legend = ax.legend(handles=legend_handles, bbox_to_anchor=(0, -0.1, 1.8, 0.2),
                    loc='lower center', bbox_transform=ax.transAxes,
                    prop=fontprops, ncol=8, handlelength=1.5, mode='expand', columnspacing=0.8, frameon=False)

# 调整布局
plt.tight_layout()
# 显示图例（可选）
plt.show()

In [ ]:
# --------------------------------------- 计算流域尺度时间评分，导出到表格 ------------------------------
# 读取数据
Basin_db_filename_TotalSMdata = f"merged_test_SM_withBASIN_{MODEL_NAME}_{SOIL_DEPTH}_GDS_Daily_AMSR_FY.csv"
Basin_db_file = os.path.join(DatabasePath  ,Basin_db_filename_TotalSMdata)
Basin_db_df_TotalSMdata = pd.read_csv(Basin_db_file, dtype={'ID': str})
Basin_db_df_TotalSMdata['Date'] = pd.to_datetime(Basin_db_df_TotalSMdata['Date'])
print(Basin_db_df_TotalSMdata.columns)


Subset_SM_datas=[ "CAMSR*", "GAMSR*", "GDS-Daily", "FY*", "ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]# SM_datas=[ "CAMSR*", "GAMSR*", "GDS_Daily", "FY", "ERA5" ,"ERA5-Land" , "GLDAS-Noah", "GLDAS-CLSM", "SMCI*" ,"CLDAS*" ,MODEL_NAME]

for METRIC in ["KGE"]:
    # 初始化结果DataFrame
    columns = Subset_SM_datas
    # index = ["Song-Liao","Inland","Yellow","Haihe", "Huaihe", "Yangtze", "Southwest",  "Southeast", "Pearl" ,"CN"]
    result_df = pd.DataFrame(index=BASINS, columns=columns)
    for idx, Basin in enumerate(BASINS):  
        if Basin=="CN":
            Basin_SM_df = Basin_db_df_TotalSMdata.copy()
            # print(db_df)
        else:
            Basin_SM_df = Basin_db_df_TotalSMdata[Basin_db_df_TotalSMdata['BASIN']==Basin]
        # --------------------------计算评分------------------------------
        # 计算
        if METRIC=='RMSE':
            Trmse_df = (
                Basin_SM_df.groupby(id_vars, as_index=False)
                .apply(lambda group: calculate_Trmse(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=30))
                .reset_index(drop=True)
            )  # 均方根误差
            METRICdf =  Trmse_df.copy()
        elif METRIC=='CORR':
            Tcorr_df = (
                Basin_SM_df.groupby(id_vars, as_index=False)
                .apply(lambda group: calculate_Tcorr(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=30))
                .reset_index(drop=True)
            )  # 相关系数
            METRICdf =  Tcorr_df.copy() 
        elif METRIC=='BIAS':
            Tbias_df = (
                Basin_SM_df.groupby(id_vars, as_index=False)
                .apply(lambda group: calculate_Tbias(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=30))
                .reset_index(drop=True)
            )  # 偏差
            METRICdf =  Tbias_df.copy() 
        elif METRIC=="KGE":
            # 将计算KGE的函数应用于groupby对象
            Tkge_df = (
                Basin_SM_df.groupby(id_vars, as_index=False)
                .apply(lambda group: calculate_Tkge(SM_datas=Subset_SM_datas, group=group, SampleLengthThreshold=10))
                .reset_index(drop=True)
            )  # KGE指数
            # 要筛选的列名
            columns_to_filter = Subset_SM_datas
            # 创建一个布尔索引，用于标记哪些行需要保留
            mask = (Tkge_df[columns_to_filter] >= -1).all(axis=1)
            # 根据布尔索引筛选数据框
            filtered_df = Tkge_df[mask]
            # 将KGE结果保存到 METRICdf
            METRICdf = filtered_df.copy()
            
        # 将不同流域的评分统一到一个dataframe当中    
        METRICdf = METRICdf.drop(columns=['ID'])    
        result = METRICdf.mean(axis=0)
        # 存储结果到DataFrame
        result_df.loc[Basin] = result.round(4)
        
    # 定义样式函数
    # print(result_df)
    from openpyxl.styles import Font, PatternFill
     # 保存为 CSV 文件
    result_df.to_excel(os.path.join(DatabasePath, f"{METRIC}_{MODEL_NAME}_{SOIL_DEPTH}.xlsx"))
    
    # 保存为 Excel 文件并应用样式
    output_path = os.path.join(DatabasePath, f"{METRIC}_{MODEL_NAME}.xlsx")
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        result_df.to_excel(writer, sheet_name='Sheet1')
        # 获取 Excel 工作簿和工作表
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        # 设置表头字体为 Times New Roman 并加粗
        header_font = Font(name='Times New Roman', bold=True)
        for col in range(1, len(result_df.columns) + 2):
            cell = worksheet.cell(row=1, column=col)
            cell.font = header_font
        # 设置数据字体为 Times New Roman
        data_font = Font(name='Times New Roman')
        for row in range(2, len(result_df) + 2):
            for col in range(1, len(result_df.columns) + 2):
                cell = worksheet.cell(row=row, column=col)
                cell.font = data_font
        # 标记最优值和次优值
        for row in range(2, len(result_df) + 2):
            row_values = [worksheet.cell(row=row, column=col).value for col in range(2, len(result_df.columns) + 2)]
            # 找到最优值和次优值
            sorted_values = sorted(row_values, reverse=True)
            max_val = sorted_values[0]  # 最优值
            second_max_val = sorted_values[1]  # 次优值
            for col in range(2, len(result_df.columns) + 2):
                cell = worksheet.cell(row=row, column=col)
                if cell.value == max_val:
                    cell.font = Font(name='Times New Roman', bold=True, color='FF0000')  # 红色加粗
                elif cell.value == second_max_val:
                    cell.font = Font(name='Times New Roman', bold=True)  # 黑色加粗

result_df